# Exploratory Data Analysis

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="ticks", color_codes=True)
import pylab as pl

df = pd.read_csv('../data/all_merged_data.csv')
CMs = df.columns[5:]
countries = ['Austria', 'UK', 'Germany', 'Italy', 'Czech', 'Switzerland', 'Netherlands']

## Binning the limit NPIs

In [ ]:
def make_bin(df, npi, limit):
    data = df[npi]
    rows = [i for i in range(len(data)) if data[i] > 0 and data[i] <= limit]
    new_data = np.zeros_like(data)
    new_data[rows] = 1
    col_name = f'{npi}ed to {limit} or less'
    df[col_name] = new_data
    return df

gathering_npis = [CMs[i] for i in [0,1,2,3]]
household_npis = [CMs[i] for i in [4,5,6,7]]

def make_all_bins(df, gathering_lims, household_lims):
    data = df
    for npi in gathering_npis:
        for lim in gathering_lims:
            data = make_bin(data, npi, lim)
    for npi in household_npis:
        for lim in household_lims:
            data = make_bin(data, npi, lim)
    return data

In [ ]:
gathering_limits = [1, 5, 10, 30, 200, 1000]
household_limits = [1, 2]

df = make_all_bins(df, gathering_limits, household_limits)
df = df.drop(columns=CMs[:8])
CMs = df.columns[5:]

## NPI representation by region


In [ ]:
def count_active_days_region(npi, country):
    df_country = df[df['Country']==country]
    #non_zero_rows = [ind for ind in range(df_country.shape[0]) if df_country[npi][ind] != 0]
    non_zero_rows = df_country[df_country[npi] != 0]
    regions = np.unique(df_country['Area'])
    active_inds = []
    for region in regions:
        #active_ind = [ind for ind in non_zero_rows if df_country['Area'][ind] == region]
        #active_ind = df_country.loc[df_country['Area'] == region]
        active_rows = non_zero_rows[non_zero_rows['Area']==region]
        count = active_rows.shape[0]
        active_inds.append(count)
    return regions, active_inds

active_days_uk = [count_active_days_region(npi, 'England')[1] for npi in CMs]
active_days_de = [count_active_days_region(npi, 'Germany')[1] for npi in CMs]
active_days_at = [count_active_days_region(npi, 'Austria')[1] for npi in CMs]
active_days_it = [count_active_days_region(npi, 'Italy')[1] for npi in CMs]
active_days_cz = [count_active_days_region(npi, 'Czech')[1] for npi in CMs]
active_days_ch = [count_active_days_region(npi, 'Switzerland')[1] for npi in CMs]
active_days_nl = [count_active_days_region(npi, 'Netherlands')[1] for npi in CMs]
# summing up counts from the regions for each npi:
region_counts_uk = [sum(i) for i in active_days_uk]
region_counts_de = [sum(i) for i in active_days_de]
region_counts_at = [sum(i) for i in active_days_at]
region_counts_it = [sum(i) for i in active_days_it]
region_counts_cz = [sum(i) for i in active_days_cz]
region_counts_ch = [sum(i) for i in active_days_ch]
region_counts_nl = [sum(i) for i in active_days_nl]

In [ ]:
region_counts = pd.DataFrame((region_counts_at, region_counts_uk,
region_counts_de, region_counts_it, region_counts_cz, region_counts_ch, region_counts_nl)).T.set_index(CMs)
region_counts.columns = countries
region_counts.plot(kind='bar', stacked=True, title='Coloured bar length is the sum of days across the country\'s regions that the NPI is active', figsize = (10,5))
plt.show()
region_counts_normalised = region_counts.div(region_counts.sum(axis=1), axis=0)
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(region_counts_normalised, xticklabels = region_counts.columns, yticklabels = CMs, ax=ax).set_title('Active days attributable to each country')

In [ ]:
def plot_region_contribution_heatmap(active_days_specific_country, country):
    active_days_specific_country = pd.DataFrame(active_days_specific_country)
    active_days_specific_country.columns = count_active_days_region(CMs[0], country)[0]
    active_days_specific_country.index = CMs
    active_days_normalised = active_days_specific_country.div(active_days_specific_country.sum(axis=1), axis=0)
    fig, ax = plt.subplots(figsize=(10,10))
    return sns.heatmap(active_days_normalised, xticklabels = active_days_specific_country.columns, yticklabels = CMs, ax = ax).set_title('Proportion of active days in '+ country + ' attributable to each constituent region')

In [ ]:
plot_region_contribution_heatmap(active_days_uk, 'England')

In [ ]:
plot_region_contribution_heatmap(active_days_de, 'Germany')

In [ ]:
plot_region_contribution_heatmap(active_days_at, 'Austria')

In [ ]:
plot_region_contribution_heatmap(active_days_it, 'Italy')

In [ ]:
plot_region_contribution_heatmap(active_days_cz, 'Czech')

In [ ]:
plot_region_contribution_heatmap(active_days_ch, 'Switzerland')

In [ ]:
plot_region_contribution_heatmap(active_days_nl, 'Netherlands')

## Thresholding

In [ ]:
# thresholding_CMs = [CM for CM in CMs if len(np.unique(df[CM]))>2]
# df_reduced = df
# df_thresholds = df_reduced[thresholding_CMs]
# fig = plt.figure(figsize=((15,10)))
# fig.subplots_adjust(hspace=1, wspace=1)
# for i in range(len(thresholding_CMs)):
#     ax = fig.add_subplot(3, 3, i+1)
#     plt.xticks(rotation=90)
#     sns.countplot(x = thresholding_CMs[i], data=df_thresholds, ax=ax)
# pl.suptitle("Histogram of values (thresholds) for the relevant NPIs")
# plt.show()

## Correlations

Correlations don't work when you have an npi always inactive or active for a region,
so instead this is a plot of proportion of region-days where the state of each npi is the same.

In [ ]:
def corr_region(region):
    df_region = df[df['Area'] == region].drop(columns=['Area', 'Date', 'Country', 'New Cases', 'New Deaths'])
    df_binary = (df_region > 0).astype(float)
    corr_mat = np.zeros((len(CMs), len(CMs)))
    for i in range(len(CMs)):
        for j in range(len(CMs)):
            corr_mat[i, j] = np.sum(df_binary[CMs[i]] == df_binary[CMs[j]])/df_binary.shape[0]
    return corr_mat
def corr_country(country):
    df_country = df[df['Country'] == country]
    regions = list(set(df_country['Area']))
    return np.nanmean([corr_region(region) for region in regions], axis=0)
def corr_total():
    countries = list(set(df['Country']))
    return np.nanmean([corr_country(country) for country in countries], axis=0)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(pd.DataFrame(corr_total(), dtype=float), xticklabels = CMs, yticklabels = CMs).set_title('Correlation plot, aggregated across countries')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(pd.DataFrame(corr_country('Austria'), dtype=float), xticklabels = CMs, yticklabels = CMs, ax = ax).set_title('Austria: Correlation plot')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(pd.DataFrame(corr_country('England'), dtype=float), xticklabels = CMs, yticklabels = CMs, ax = ax).set_title('England: Correlation plot')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(pd.DataFrame(corr_country('Germany'), dtype=float), xticklabels = CMs, yticklabels = CMs, ax = ax).set_title('Germany: Correlation plot')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(pd.DataFrame(corr_country('Italy'), dtype=float), xticklabels = CMs, yticklabels = CMs, ax = ax).set_title('Italy: Correlation plot')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(pd.DataFrame(corr_country('Czech'), dtype=float), xticklabels = CMs, yticklabels = CMs, ax = ax).set_title('Czech: Correlation plot')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(pd.DataFrame(corr_country('Switzerland'), dtype=float), xticklabels = CMs, yticklabels = CMs, ax = ax).set_title('Switzerland: Correlation plot')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(pd.DataFrame(corr_country('Netherlands'), dtype=float), xticklabels = CMs, yticklabels = CMs, ax = ax).set_title('Netherlands: Correlation plot')
plt.show()

## Conditional activation


In [ ]:
def get_active_dates_region(data, npi):
    non_zero_dates = data['Date'][data[npi] != 0]
    return non_zero_dates

def intersect_npi_active_dates_region(active_days_1, active_days_2):
    return set(active_days_1).intersection(set(active_days_2))

def proportional_activation_region(active_days_1, active_days_2):
    number_of_active_days = len(active_days_1)
    if number_of_active_days == 0: return float('nan')
    number_of_jointly_active_days = len(intersect_npi_active_dates_region(active_days_1, active_days_2))
    return number_of_jointly_active_days/number_of_active_days

def activation_difference_region(active_days_1, active_days_2):
    return len(set(active_days_1) - set(active_days_2))

def only_one_active_region(active_days_1, active_days_2):
    return len(set(active_days_1) - set(active_days_2)) + len(set(active_days_2) - set(active_days_1))

def find_conditional_activations(country):
    regions = np.unique(df['Area'][df['Country']==country])
    proportional_activations = np.empty((len(CMs),len(CMs)), dtype=object)
    activation_differences = np.empty((len(CMs),len(CMs)), dtype=object)
    single_activations = np.empty((len(CMs),len(CMs)), dtype=object)
    count = 0
    for region in regions:
        data = df[df['Area']==region]
        for i in range(len(CMs)):
            for j in range(len(CMs)):
                active_days_1 = get_active_dates_region(data, CMs[i])
                active_days_2 = get_active_dates_region(data, CMs[j])
                if count == 0:
                    proportional_activations[i, j] = [proportional_activation_region(active_days_1, active_days_2)]
                    activation_differences[i, j] = [activation_difference_region(active_days_1, active_days_2)]
                    single_activations[i, j] = [only_one_active_region(active_days_1, active_days_2)]
                else:
                    proportional_activations[i, j].append(proportional_activation_region(active_days_1, active_days_2))
                    activation_differences[i, j].append(activation_difference_region(active_days_1, active_days_2))
                    single_activations[i, j].append(only_one_active_region(active_days_1, active_days_2))
        count += 1
    for i in range(len(CMs)):
        for j in range(len(CMs)):
            proportional_activations[i, j] = np.mean(proportional_activations[i, j])
            activation_differences[i, j] = np.sum(activation_differences[i, j])
            single_activations[i, j] = np.sum(single_activations[i, j])

    return proportional_activations, activation_differences, single_activations

def find_conditional_activations_total():

    proportional_activations = np.empty((len(CMs),len(CMs)), dtype=object)
    activation_differences = np.empty((len(CMs),len(CMs)), dtype=object)
    single_activations = np.empty((len(CMs),len(CMs)), dtype=object)

    countries = np.unique(df['Country'])
    count = 0
    for country in countries:
        prop, act, single = find_conditional_activations(country)
        for i in range(len(CMs)):
            for j in range(len(CMs)):
                if count == 0:
                    proportional_activations[i, j] = [prop[i,j]]
                    activation_differences[i, j] = [act[i,j]]
                    single_activations[i, j] = [single[i,j]]
                else:
                    proportional_activations[i, j].append(prop[i,j])
                    activation_differences[i, j].append(act[i,j])
                    single_activations[i, j].append(single[i,j])
        count += 1
    for i in range(len(CMs)):
        for j in range(len(CMs)):
            proportional_activations[i, j] = np.nanmean(proportional_activations[i, j])
            activation_differences[i, j] = np.sum(activation_differences[i, j])
            single_activations[i, j] = np.sum(single_activations[i, j])

    return proportional_activations, activation_differences, single_activations

In [ ]:
prop, act, single = find_conditional_activations_total()
fig = plt.figure(figsize=((10,40)))
fig.subplots_adjust(hspace=1)
ax = fig.add_subplot(3, 1, 1)
sns.heatmap(pd.DataFrame(prop, dtype=float), xticklabels = CMs, yticklabels = CMs).set_title(': Conditional activation matrix: of the proportion of time that NPI Y was active, how often was NPI X active?')
ax = fig.add_subplot(3, 1, 2)
sns.heatmap(pd.DataFrame(act, dtype=float), xticklabels = CMs, yticklabels = CMs).set_title(': Number of days where only NPI Y is active')
ax = fig.add_subplot(3, 1, 3)
sns.heatmap(pd.DataFrame(single, dtype=float), xticklabels = CMs, yticklabels = CMs).set_title(': Number of days where exactly one of NPI X and NPI Y is active')
plt.show()

In [ ]:
def conditional_activation_plotter(country):
    prop, act, single = find_conditional_activations(country)
    fig = plt.figure(figsize=((10,40)))
    fig.subplots_adjust(hspace=1)
    ax = fig.add_subplot(3, 1, 1)
    sns.heatmap(pd.DataFrame(prop, dtype=float), xticklabels = CMs, yticklabels = CMs).set_title(country + ': Conditional activation matrix: of the proportion of time that NPI Y was active, how often was NPI X active?')
    ax = fig.add_subplot(3, 1, 2)
    sns.heatmap(pd.DataFrame(act, dtype=float), xticklabels = CMs, yticklabels = CMs).set_title(country + ': Number of days where only NPI Y is active')
    ax = fig.add_subplot(3, 1, 3)
    sns.heatmap(pd.DataFrame(single, dtype=float), xticklabels = CMs, yticklabels = CMs).set_title(country + ': Number of days where exactly one of NPI X and NPI Y is active')
    plt.show()


### Austria

In [ ]:
conditional_activation_plotter('Austria')

### England

In [ ]:
conditional_activation_plotter('England')

### Germany

In [ ]:
conditional_activation_plotter('Germany')

### Italy

In [ ]:
conditional_activation_plotter('Italy')

In [ ]:
conditional_activation_plotter('Czech')

In [ ]:
conditional_activation_plotter('Switzerland')

In [ ]:
conditional_activation_plotter('Netherlands')

## Paired Activation Differences

In [ ]:
def get_start_dates(data, npi, region):
    df_region = data[data['Area']==region]
    start_dates = []
    if list(df_region[npi])[0] != 0:
        start_dates.append(0)
    #loop through list and find date when go from 0 to non-zero
    last = list(df_region[npi])[0]
    for i in range(1, df_region.shape[0]):
        next = list(df_region[npi])[i]
        if last == 0 and next != 0:
            start_dates.append(i)
        last = next
    return start_dates

def compare_dates(ds1, ds2):
    diffs = []
    for d1 in ds1:
        for d2 in ds2:
            diffs.append(d1 - d2)
    return diffs

def compute_start_dates(data):
    starts = np.empty((len(CMs), len(CMs)), dtype=object)
    for npi1 in range(len(CMs)):
        for npi2 in range(len(CMs)):
            starts[npi1, npi2] = []
            for region in np.unique(data['Area']):
                starts[npi1, npi2].append(compare_dates(get_start_dates(data, CMs[npi1], region),get_start_dates(data, CMs[npi2], region)))
            starts[npi1, npi2] = [diff for diffs in starts[npi1, npi2] for diff in diffs]
    return starts

def extract_diff_population(difference_data, threshold):
    count_data = np.zeros_like(difference_data)
    for i in range(count_data.shape[0]):
        for j in range(count_data.shape[1]):
            diff_reduced = [diff for diff in difference_data[i, j] if abs(diff) < threshold]
            count_data[i,j] = len(diff_reduced)
    return count_data

def get_end_dates(data, npi, region):
    df_region = data[data['Area']==region]
    end_dates = []
    #loop through list and find date when go from 0 to non-zero
    last = list(df_region[npi])[0]
    for i in range(1, df_region.shape[0]):
        next = list(df_region[npi])[i]
        if last != 0 and next == 0:
            end_dates.append(i)
        last = next
    return end_dates

def compute_end_dates(data):
    ends = np.empty((len(CMs), len(CMs)), dtype=object)
    for npi1 in range(len(CMs)):
        for npi2 in range(len(CMs)):
            ends[npi1, npi2] = []
            for region in np.unique(data['Area']):
                ends[npi1, npi2].append(compare_dates(get_end_dates(data, CMs[npi1], region),get_end_dates(data, CMs[npi2], region)))
            ends[npi1, npi2] = [diff for diffs in ends[npi1, npi2] for diff in diffs]
    return ends

In [ ]:
starts = compute_start_dates(df)
# np.save('../data/npi_paired_activations', starts)

In [ ]:
ends = compute_end_dates(df)
# np.save('../data/npi_paired_deactivations', ends)

In [ ]:
starts_extracted = extract_diff_population(starts, 0.5)
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(pd.DataFrame(starts_extracted, dtype=float), xticklabels = CMs, yticklabels = CMs, ax=ax).set_title('On how many days (across regions) did each NPI pair activate at exactly the same time?')

In [ ]:
starts_extracted_5 = extract_diff_population(starts, 5)
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(pd.DataFrame(starts_extracted_5, dtype=float), xticklabels = CMs, yticklabels = CMs, ax=ax).set_title('On how many days (across regions) did each NPI pair activate within 5 days of each other?')

In [ ]:
ends_extracted = extract_diff_population(ends, 0.5)
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(pd.DataFrame(ends_extracted, dtype=float), xticklabels = CMs, yticklabels = CMs, ax=ax).set_title('On how many days (across regions) did each NPI pair deactive at exactly the same time?')

In [ ]:
ends_extracted_5 = extract_diff_population(ends, 5)
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(pd.DataFrame(ends_extracted_5, dtype=float), xticklabels = CMs, yticklabels = CMs, ax=ax).set_title('On how many days (across regions) did each NPI pair deactive within 5 days of each other?')

### Country-specific joint activation/deactivation

In [ ]:
df_uk = df[df['Country']=='England']
df_de = df[df['Country']=='Germany']
df_it = df[df['Country']=='Italy']
df_at = df[df['Country']=='Austria']
df_cz = df[df['Country']=='Czech']
df_ch = df[df['Country']=='Switzerland']

In [ ]:
# starts_uk = compute_start_dates(df_uk)
# ends_uk = compute_end_dates(df_uk)
# starts_de = compute_start_dates(df_de)
# ends_de = compute_end_dates(df_de)
# starts_it = compute_start_dates(df_it)
# ends_it = compute_end_dates(df_it)
# starts_at = compute_start_dates(df_at)
# ends_at = compute_end_dates(df_at)
# starts_cz = compute_start_dates(df_cz)
# ends_cz = compute_end_dates(df_cz)
starts_ch = compute_start_dates(df_ch)
ends_ch = compute_end_dates(df_ch)
starts_countries = [starts_at, starts_uk, starts_de, starts_it, starts_cz, starts_ch]
ends_countries = [ends_at, ends_uk, ends_de, ends_it, ends_cz, ends_ch]

In [ ]:
def plot_joint_activation_heatmap(data, country, threshold, activate, ax):
    extracted = extract_diff_population(data, threshold)
    sns.heatmap(pd.DataFrame(extracted, dtype=float), xticklabels = CMs, yticklabels = CMs, ax=ax).set_title('On how many days across regions in '+ country + ' did each NPI pair '+activate+' within '+ str(threshold) + ' days of one another?')
def plot_all_joint_activation_heatmaps(threshold, activate = 'activate'):
    fig = plt.figure(figsize=((10,80)))
    fig.subplots_adjust(hspace=1)
    fig_number = 0
    for country in np.unique(df['Country']):
        fig_number += 1
        ax = fig.add_subplot(len(countries), 1, fig_number)
        if activate=='activate':
            data = starts_countries
        else:
            data = ends_countries
        plot_joint_activation_heatmap(data[fig_number-1], country, threshold, activate, ax = ax)
    plt.show()

In [ ]:
plot_all_joint_activation_heatmaps(0.5)

In [ ]:
plot_all_joint_activation_heatmaps(0.5, 'deactivate')
